# Naive Bayes Classifier with NLTK


In [5]:
import nltk
import random
from nltk.corpus import movie_reviews

documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

all_words = []

for w in movie_reviews.words():
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:3000]

In [6]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [7]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

Now it is time to choose an algorithm, separate our data into training and testing sets, and press go! The algorithm that we're going to use first is the Naive Bayes classifier. This is a pretty popular algorithm used in text classification, so it is only fitting that we try it out first. Before we can train and test our algorithm, however, we need to go ahead and split up the data into a training set and a testing set.

You could train and test on the same dataset, but this would present you with some serious bias issues, so you should never train and test against the exact same data. To do this, since we've shuffled our data set, we'll assign the first 1,900 shuffled reviews, consisting of both positive and negative reviews, as the training set. Then, we can test against the last 100 to see how accurate we are.

This is called supervised machine learning, because we're showing the machine data, and telling it "hey, this data is positive," or "this data is negative." Then, after that training is done, we show the machine some new data and ask the computer, based on what we taught the computer before, what the computer thinks the category of the new data is.

We can split the data with:



In [9]:
# set that we'll train our classifier with
training_set = featuresets[:1900]

# set that we'll test against.
testing_set = featuresets[1900:]

Next, we can define, and train our classifier like:



In [20]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

First we just simply are invoking the Naive Bayes classifier, then we go ahead and use .train() to train it all in one line.

Easy enough, now it is trained. Next, we can test it:

In [21]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier,testing_set))*100)

Classifier accuracy percent: 73.0


Next, we can take it a step further to see what the most valuable words are when it comes to positive or negative reviews:


In [23]:
classifier.show_most_informative_features(15)

Most Informative Features
               ludicrous = True              neg : pos    =      9.7 : 1.0
               furniture = True              neg : pos    =      7.5 : 1.0
                 supreme = True              pos : neg    =      7.1 : 1.0
               behaviour = True              pos : neg    =      7.1 : 1.0
                poignant = True              pos : neg    =      6.7 : 1.0
               integrity = True              pos : neg    =      6.5 : 1.0
                    noah = True              pos : neg    =      6.5 : 1.0
                  sloppy = True              neg : pos    =      6.2 : 1.0
                     bio = True              neg : pos    =      6.2 : 1.0
                  elmore = True              pos : neg    =      5.8 : 1.0
                    dice = True              neg : pos    =      5.6 : 1.0
                   inept = True              neg : pos    =      5.3 : 1.0
                fondness = True              pos : neg    =      5.1 : 1.0

What this tells you is the ratio of occurences in negative to positive, or visa versa, for every word. So here, we can see that the term "ludicrous" appears 9.7 more times as often in negative reviews as it does in positive reviews. 

Now, let's say you were totally content with your results, and you wanted to move forward, maybe using this classifier to predict things right now. It would be very impractical to train the classifier, and retrain it every time you needed to use it. As such, you can save the classifier using the pickle module. Let's do that next.


# Saving Classifiers with NLTK

Training classifiers and machine learning algorithms can take a very long time, especially if you're training against a larger data set. Ours is actually pretty small. Can you imagine having to train the classifier every time you wanted to fire it up and use it? What horror! Instead, what we can do is use the Pickle module to go ahead and serialize our classifier object, so that all we need to do is load that file in real quick.

So, how do we do this? The first step is to save the object. To do this, first you need to import pickle at the top of your script, then, after you have trained with .train() the classifier, you can then call the following lines:



In [26]:
import pickle

In [27]:
save_classifier = open("naivebayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

This opens up a pickle file, preparing to write in bytes some data. Then, we use pickle.dump() to dump the data. The first parameter to pickle.dump() is what are you dumping, the second parameter is where are you dumping it.

After that, we close the file as we're supposed to, and that is that, we now have a pickled, or serialized, object saved in our script's directory!

Next, how would we go about opening and using this classifier? The .pickle file is a serialized object, all we need to do now is read it into memory, which will be about as quick as reading any other ordinary file. To do this:

In [28]:
classifier_f = open("naivebayes.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

Here, we do a very similar process. We open the file to read as bytes. Then, we use pickle.load() to load the file, and we save the data to the classifier variable. Then we close the file, and that is that. We now have the same classifier object as before!

Now, we can use this object, and we no longer need to train our classifier every time we wanted to use it to classify.

While this is all fine and dandy, we're probably not too content with the 60-75% accuracy we're getting. What about other classifiers? Turns out, there are many classifiers, but we need the scikit-learn (sklearn) module. Luckily for us, the people at NLTK recognized the value of incorporating the sklearn module into NLTK, and they have built us a little API to do it. That's what we'll be doing in the next